In [1]:
import pandas as pd
import time
from datetime import datetime
import math
import ccxt
import json

In [2]:
with open('binance.json') as f:
    binance_api = json.load(f)

In [3]:
exchange_id = 'binance'
exchange_class = getattr(ccxt, exchange_id)

exchange = exchange_class({
    "apiKey": binance_api['apiKey'],
    "secret": binance_api['secret'],
})

In [4]:
markets = exchange.fetchMarkets()
market_symbols = [market['symbol'] for market in markets if market['type']=='spot']

In [5]:
def create_chain(base_currency, token1, token2, buy_t1, buy_t2_by_t1, sell_t2, res):   
    return {
        'base': base_currency, # USDT                 
        'token1':token1,      # BTC                 
        'token2':token2,      # ETH       
        'buy_t1': buy_t1,     
        'buy_t2_by_t1': buy_t2_by_t1,       
        'sell_t2': sell_t2,
        'Result' : res
    }
    

def get_chains(market_symbols, quotes, base_currency='USDT'):
    chains = []

    for symbol in market_symbols:
        try:
            token1 = symbol.split('/')[0] 
            token2 = symbol.split('/')[1]
            #ищем промежуточное звено треугольника и проверяем результат в 2 стороны
            if (token1!=base_currency and token2!=base_currency):
                start = quotes[f'{token1}/{base_currency}']
                middle = quotes[f'{token1}/{token2}']
                finish = quotes[f'{token2}/{base_currency}']
                if  start['ask']!=0 and start['bid']!=0 and middle['ask']!=0 and middle['bid']!=0 and finish['ask']!=0 and finish['bid']!=0:
                    #USDT-ETH-BTC
                    chains.append(
                                create_chain(
                                    base_currency, #USDT
                                    token1,        #ETH
                                    token2,        #BTC
                                    start['ask'],  #ETH/USDT ASK
                                    middle['bid'], #ETH/BTC BID
                                    finish['bid'], #BTC/USDT BID
                                    float(100.0*middle['bid']*finish['bid']/start['ask']-100.0)
                                )
                    )
                    #USDT-BTC-ETH
                    chains.append(
                                create_chain(
                                    base_currency, #USDT
                                    token2,        #BTC
                                    token1,        #ETH
                                    finish['ask'], #BTC/USDT ASK
                                    middle['ask'], ##ETH/BTC ASK
                                    start['bid'],  #ETH/USDT BID
                                    float(100.0*start['bid']/finish['ask']/middle['ask']-100.0)
                                )
                   )

        except KeyError:
            pass
    return pd.DataFrame(chains)


In [ ]:
while(1):    
    quotes = exchange.fetchBidsAsks()
    chains = get_chains(market_symbols, quotes, 'USDT')
    sorted_chains = chains.sort_values(by='Result', ascending=False)
    profit = sorted_chains.loc[sorted_chains['Result']>0.3]
    if not profit.empty:
        print(f"Time-{datetime.now().strftime('%H:%M:%S')}")
        print(profit)
    time.sleep(1)

Time-15:38:02
      base token1 token2  buy_t1  buy_t2_by_t1  sell_t2    Result
1214  USDT   BOND   BUSD   5.474         5.493   1.0002  0.367165
Time-15:38:24
      base token1 token2  buy_t1  buy_t2_by_t1  sell_t2    Result
1665  USDT    BNB   GLMR   235.9      0.002507   0.5941  0.456323
Time-15:44:23
     base token1 token2   buy_t1  buy_t2_by_t1  sell_t2    Result
357  USDT    BTC   PERL  20625.2      0.000001  0.02283  0.627123
Time-15:47:35
     base token1 token2  buy_t1  buy_t2_by_t1  sell_t2    Result
912  USDT   DEXE   BUSD    2.98          2.99   1.0002  0.355638
Time-15:47:36
     base token1 token2  buy_t1  buy_t2_by_t1  sell_t2    Result
912  USDT   DEXE   BUSD   2.974          2.99   1.0002  0.558104


In [29]:
#token1 - token2
100.0*0.674300*1.0008/0.67410-100.0

0.10969292389852114

In [30]:
#token2 - token1
100.0*0.513/19727.1600/0.000026-100.0

0.018295744391153335